In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import csv

# 1. Prepare the dataset
class CustomDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_length):
      self.texts = texts
      self.labels = labels
      self.tokenizer = tokenizer
      self.max_length = max_length

  def __len__(self):
      return len(self.texts)

  def __getitem__(self, idx):
      text = self.texts[idx]
      label = self.labels[idx]

      encoding = self.tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_length,
          return_token_type_ids=False,
          padding='max_length',
          truncation=True,
          return_attention_mask=True,
          return_tensors='pt',
      )

      return {
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(label, dtype=torch.long)
      }
# 2. Load and preprocess the data
# Load the TSV file
df = pd.read_csv('/Users/aaravnoronha/Desktop/Student Feedback Data - Training Model.tsv', sep='\t')
# Extract Feedback and Rating
texts = df['Feedback'].tolist()
ratings = df['Rating'].tolist()
def convert_rating_to_label(rating):
  try:
      rating = float(rating)
      if rating <= 1.0:
          return 0
      elif rating <= 2.0:
          return 1
      elif rating <= 3.0:
          return 2
      elif rating <= 4.0:
          return 3
      else:
          return 4
  except ValueError:
      return -1  # Invalid rating

# Convert ratings to 5 labels
labels = [convert_rating_to_label(rating) for rating in ratings]
# Split the data
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Create datasets
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_length=128)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_length=128)
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
# 3. Initialize the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
# 4. Set up optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_loader) * 3  # Number of epochs
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, total_iters=total_steps)
# 5. Training loop
num_epochs = 3

for epoch in range(num_epochs):
  model.train()
  for batch in train_loader:
      optimizer.zero_grad()
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      scheduler.step()

  # Validation
  model.eval()
  val_loss = 0
  correct_predictions = 0
  with torch.no_grad():
      for batch in val_loader:
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['labels'].to(device)

          outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
          loss = outputs.loss
          val_loss += loss.item()
          
          preds = torch.argmax(outputs.logits, dim=1)
          correct_predictions += torch.sum(preds == labels)

  val_accuracy = correct_predictions.double() / len(val_dataset)
  print(f"Epoch {epoch+1}/{num_epochs}")
  print(f"Validation Loss: {val_loss/len(val_loader):.4f}")
  print(f"Validation Accuracy: {val_accuracy:.4f}")

# 6. Save the model
torch.save(model.state_dict(), 'feedback_sentiment_model.pth')
print("Model saved successfully.")
def load_model(model_path):
  # Load the trained model
  model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
  model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
  model.eval()
  return model
    
def predict(model, tokenizer, text):
  # Tokenize the input text
  encoding = tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=128,
      return_token_type_ids=False,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',
  )

  # Move the input to the same device as the model
  input_ids = encoding['input_ids'].to(model.device)
  attention_mask = encoding['attention_mask'].to(model.device)

  # Perform inference
  with torch.no_grad():
      outputs = model(input_ids, attention_mask=attention_mask)
      _, prediction = torch.max(outputs.logits, dim=1)

  return prediction.item()
# Load the model and tokenizer
model_path = 'feedback_sentiment_model.pth'  
model = load_model(model_path)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')# Read the TSV file manually
data = []
with open('/Users/aaravnoronha/Desktop/Student Feedback Data - Training Model.tsv', 'r', encoding='utf-8') as file:
  tsv_reader = csv.reader(file, delimiter='\t')
  headers = next(tsv_reader)  # Read the header row
  for row in tsv_reader:
      if len(row) >= 2:  # Ensure we have at least Rating and Feedback
          data.append({'Rating': row[0], 'Feedback': row[1]})

# Convert to DataFrame
df = pd.DataFrame(data)
def get_label(rating):
  try:
      rating = float(rating)
      if rating <= 1.0:
          return 0
      elif rating <= 2.0:
          return 1
      elif rating <= 3.0:
          return 2
      elif rating <= 4.0:
          return 3
      else:
          return 4
  except ValueError:
      return -1  # Invalid rating
          
def get_rating_from_label(label):
  ratings = [0.5, 1.5, 2.5, 3.5, 4.5]
  return ratings[label]
print("Predictions for all student feedback entries:")
total = 0
correct = 0
for i, row in df.iterrows():
  rating = row['Rating']
  feedback = row['Feedback']
  
  if pd.isna(feedback) or feedback.strip() == '':
      continue
  
  actual_label = get_label(rating)
  predicted_label = predict(model, tokenizer, str(feedback))
  
  print(f"Feedback {i+1}:")
  print(f"  Rating: {rating}")
  print(f"  Feedback: {feedback}")
  print(f"  Actual Label: {actual_label} (Rating: {get_rating_from_label(actual_label)})")
  print(f"  Predicted Label: {predicted_label} (Rating: {get_rating_from_label(predicted_label)})")
  print()

  total += 1
  if actual_label == predicted_label:
      correct += 1

accuracy = correct / total if total > 0 else 0
print(f"Overall Accuracy: {accuracy:.2f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/aaravnoronha/Desktop/myenv/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3
Validation Loss: 0.8325
Validation Accuracy: 0.7200
Epoch 2/3
Validation Loss: 0.8421
Validation Accuracy: 0.7300
Epoch 3/3
Validation Loss: 0.6618
Validation Accuracy: 0.7600
Model saved successfully.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predictions for all student feedback entries:
Feedback 1:
  Rating: 3
  Feedback: Awesome class
  Actual Label: 2 (Rating: 2.5)
  Predicted Label: 4 (Rating: 4.5)

Feedback 2:
  Rating: 3
  Feedback: Awesome class.
  Actual Label: 2 (Rating: 2.5)
  Predicted Label: 0 (Rating: 0.5)

Feedback 3:
  Rating: 3.5
  Feedback: Great Class!!
  Actual Label: 3 (Rating: 3.5)
  Predicted Label: 4 (Rating: 4.5)

Feedback 4:
  Rating: 3
  Feedback: great class
  Actual Label: 2 (Rating: 2.5)
  Predicted Label: 4 (Rating: 4.5)

Feedback 5:
  Rating: 0.5
  Feedback: nobody showed up.
  Actual Label: 0 (Rating: 0.5)
  Predicted Label: 0 (Rating: 0.5)

Feedback 6:
  Rating: 1.5
  Feedback: Mentor wasn't present for the class. Hoping to see mentor for the next booked class :)
  Actual Label: 1 (Rating: 1.5)
  Predicted Label: 0 (Rating: 0.5)

Feedback 7:
  Rating: 0.5
  Feedback: No one show up for class today
  Actual Label: 0 (Rating: 0.5)
  Predicted Label: 0 (Rating: 0.5)

Feedback 8:
  Rating: 1
  F